In [1]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [2]:
import os
import numpy as np

from lib.parameters import Parameters
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

In [3]:
##  configuration options
el_charge = 1.6e-19
rho_beam_scale = 1
config_file = '/home/cosmonaut/mount/m37s15/parameters.xml'

cfg = Parameters(config_file)
clim_estimation = 1e6 # cfg.get_clim_estimation()

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
shape=[0, 0, cfg.number_r_grid, cfg.number_z_grid]

timestamp=1.48e-9
show_grid=True
use_cache=False
cmap='terrain'
specie='electrons'

# color limits (WARNING: clim_estimation may works incorrectly)
clim_e_r = [2e5, 6e5]
clim_e_z = [2e5, 6e5]
clim_t = [0, 40]
clim_rho = [5e16, 2e17]
image_interpolation = 'bilinear'

autoselect = True

x_axis_label = r'$\mathit{Z (m)}$'
y_axis_label = r'$\mathit{R (m)}$'
cbar_axis_label = r'$\frac{V}{m}$'
cbar_bunch_density_axis_label = r'$m^{-3}$'

e_r_plot_name = r'$\mathbf{Electrical\enspace Field\enspace Radial\enspace Component}\enspace(E_r)$'
e_z_plot_name = r'$\mathbf{Electrical\enspace Field\enspace Longitudal\enspace Component}\enspace(E_z)$'
t_plot_name = r'$\mathbf{Ion\enspace Temperature}$'
rho_plot_name = r'$\mathbf{Ion\enspace Density}$'

In [4]:
# define reader (plain reader used)
if not cfg.use_hdf5:
    reader = PlainReader(path = cfg.data_path,
                         data_root=cfg.data_root,
                         fullframe_size=[cfg.number_r_grid , cfg.number_z_grid],
                         fpds=cfg.frames_per_file, 
                         use_cache=use_cache,
                         verbose=True)
else:
    reader = H5Reader(str(os.path.join(cfg.data_path, 'data.h5')), use_cache=use_cache)
    reader.verbose = True

r_scale = (shape[2] - shape[0]) / cfg.number_r_grid
z_scale = (shape[3] - shape[1]) / cfg.number_z_grid

# define plot builder
plot = PlotBuilder(shape[3] - shape[1], shape[2] - shape[0],
                   fig_color=cfg.figure_color, 
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, 
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   
                   x_ticklabel_end=cfg.z_size * z_scale, y_ticklabel_end=cfg.r_size * r_scale,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='equal', image_interpolation=image_interpolation, guess_number_ticks=20)

In [5]:
# get data
data_r = data_z = data_beam = []

for probe in cfg.probes:
    frame = cfg.get_frame_number_by_timestamp(timestamp, probe.schedule)
    if (probe.type == 'frame') and (probe.r_start == shape[0]) and (probe.z_start == shape[1]) and(probe.r_end == shape[2]) and(probe.z_end == shape[3]):
        if probe.component == 'E_r': data_r = reader.get_frame('E_r', shape, frame)
        if probe.component == 'E_z': data_z = reader.get_frame('E_z', shape, frame)
        if probe.component == 'T' and probe.specie == specie: data_t = reader.get_frame('T/{}'.format(probe.specie), shape, frame)
        if probe.component == 'density' and probe.specie == specie: data_rho = reader.get_frame('density/{}'.format(probe.specie), shape, frame)

# try bigger frames, if autoselect enabled
if len(data_r) == 0 and autoselect:
    for probe in cfg.probes:
        probe_shape = [probe.r_start, probe.z_start, probe.r_end, probe.z_end]
        frame = cfg.get_frame_number_by_timestamp(timestamp, probe.schedule)
        if (probe.type == 'frame') and (probe_shape[0] <= shape[0]) and (probe_shape[1] <= shape[1]) and(probe_shape[2] >= shape[2]) and(probe_shape[3] >= shape[3]):
            if probe.component == 'E_r' and len(data_r) == 0: data_r = reader.get_frame('E_r', probe_shape, frame)[shape[0]:shape[2], shape[1]:shape[3]]
            if probe.component == 'E_z' and len(data_z) == 0: data_z = reader.get_frame('E_z', probe_shape, frame)[shape[0]:shape[2], shape[1]:shape[3]]
            if probe.component == 'T' and probe.specie == specie: data_t = reader.get_frame('T/{}'.format(probe.specie), shape, frame)[shape[0]:shape[2], shape[1]:shape[3]]
            if probe.component == 'density' and probe.specie == specie: data_rho = reader.get_frame('density/{}'.format(probe.specie), shape, frame)[shape[0]:shape[2], shape[1]:shape[3]]

Loading data set T/electrons/frame_[0, 0, 127, 512]/74...done
Loading data set density/electrons/frame_[0, 0, 127, 512]/74...done
Loading data set E_r/frame_[0, 0, 127, 1023]/74...done
Loading data set E_z/frame_[0, 0, 127, 1023]/74...done


In [6]:
# add subplots
plot.add_subplot_cartesian_2d(e_r_plot_name, 412, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
plot.add_subplot_cartesian_2d(e_z_plot_name, 411, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
plot.add_subplot_cartesian_2d(t_plot_name, 413, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
plot.add_subplot_cartesian_2d(rho_plot_name, 414, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# add timestamp to plot
plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.95)

# add images
data_r_abs = np.abs(data_r)
data_z_abs = np.abs(data_z)
plot.add_image(e_r_plot_name, data_r_abs, cmap=cmap, clim=clim_e_r)
plot.add_image(e_z_plot_name, data_z_abs, cmap=cmap, clim=clim_e_z)
plot.add_image(t_plot_name, data_t, cmap=cmap, clim=clim_t)
plot.add_image(rho_plot_name, data_rho, cmap='gray_r', clim=clim_rho)

# add colorbars
plot.add_colorbar(e_r_plot_name, ticks=clim_e_r, title=cbar_axis_label)
plot.add_colorbar(e_z_plot_name, ticks=clim_e_z, title=cbar_axis_label)
plot.add_colorbar(t_plot_name, ticks=clim_t, title=cbar_axis_label)
plot.add_colorbar(rho_plot_name, ticks=clim_rho, title=cbar_axis_label)

plot.show()